# Launch Site Locations Analysis 

In [129]:
# libraries 

import folium
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [130]:
# mark launch sites on map

# resource
data ='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv'
df=pd.read_csv(data)
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


In [131]:
print(df.dtypes)


FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
Class               int64
dtype: object


In [132]:

# select target sub-columns: `Launch Site`, `Lat`, `Long`, `class`

data_df = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = data_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCAFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [133]:
# to place first folium map, start location is NASA Johnson Space Center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=9)

In [134]:
# create first area at Space Center coordinate with named label

circle = folium.Circle(nasa_coordinate, radius=999, color='#0000fe', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))



# create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name

marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(16,16),
        icon_anchor=(0,0),
        html='<div style="font-size: 16; color:#0000fe;"><b>%s</b></div>' % 'JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [135]:
#  map init
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# For each launch site, draw anm object based on: Lat, Long

for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Latitude'], site['Longitude']], color='#d35405', radius=50, fill=True).add_child(folium.Popup(site['LaunchSite']))
    marker = folium.Marker(
        [site['Latitude'], site['Longitude']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(16,16),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35405;"><b>%s</b></div>' % site['LaunchSite'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [136]:
# mark success/failed rateo for launchSite on the map
data_df.tail(10)

,LaunchSite,Latitude,Longitude,Class
80,CCAFS SLC 40,28.561857,-80.577366,1
81,CCAFS SLC 40,28.561857,-80.577366,1
82,CCAFS SLC 40,28.561857,-80.577366,1
83,CCAFS SLC 40,28.561857,-80.577366,1
84,CCAFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608058,-80.603956,1
86,KSC LC 39A,28.608058,-80.603956,1
87,KSC LC 39A,28.608058,-80.603956,1
88,CCAFS SLC 40,28.561857,-80.577366,1
89,CCAFS SLC 40,28.561857,-80.577366,1


In [137]:
# assign arbitrary markers, success = 1 ; failure = 0

marker_cluster = MarkerCluster()
def marker_color(launch_outcome):
   return 'green' if launch_outcome == 1 else 'red'
  

data_df['marker_color'] = data_df['Class'].apply(marker_color)
data_df.tail(20)

/var/folders/3v/sy40fy1567x17kcqbmp_5cth0000gn/T/ipykernel_4952/2378876888.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['marker_color'] = data_df['Class'].apply(marker_color)


,LaunchSite,Latitude,Longitude,Class,marker_color
70,CCAFS SLC 40,28.561857,-80.577366,1,green
71,CCAFS SLC 40,28.561857,-80.577366,1,green
72,KSC LC 39A,28.608058,-80.603956,0,red
73,CCAFS SLC 40,28.561857,-80.577366,1,green
74,CCAFS SLC 40,28.561857,-80.577366,0,red
75,CCAFS SLC 40,28.561857,-80.577366,1,green
76,KSC LC 39A,28.608058,-80.603956,0,red
77,KSC LC 39A,28.608058,-80.603956,1,green
78,KSC LC 39A,28.608058,-80.603956,1,green
79,CCAFS SLC 40,28.561857,-80.577366,1,green


In [138]:
# add markers to current map
site_map.add_child(marker_cluster)

# for each row in data frame create a marker coordinate + success or fail, 

for index, record in data_df.iterrows():
    # create & add a marker to map
    marker = folium.Marker([record['Latitude'], record['Longitude']], icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)
site_map

In [139]:
# check distance between sites to its proximities

# add mouse_position detect to get the coordinate (Lat, Long) for a mouse hover
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [140]:
# check proximities (highway, coastline etc)

from math import sin, cos, sqrt, atan2, radians

def calc_distance(lat1, lon1, lat2, lon2):
    # approx Radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [141]:
# distance to coastline

coordinates = [
    [28.56342, -80.57674],
    [28.56342, -80.56756]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calc_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])

distance_circle = folium.Marker(
    [28.56342, -80.56794],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 13; color:#d35405;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

site_map.add_child(distance_circle)
site_map

In [142]:
#d istance to Highway

coordinates = [
    [28.56342, -80.57674],
    [28.411780, -80.820630]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calc_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])

distance_circle = folium.Marker(
    [28.411780, -80.820630],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 13; color:#262629;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

site_map.add_child(distance_circle)
site_map

In [143]:
# draw a polyLine between site and arbitrary coastline

site_map.add_child(lines)

In [145]:
# distance to Florida City 

coordinates = [
    [28.56342, -80.57674],
    [28.5383, -81.3792]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calc_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])

distance_circle = folium.Marker(
    [28.5383, -81.3792],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#262629;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

site_map.add_child(distance_circle)
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

Are launch sites in proximity to railways? N
Are launch sites in proximity to highways? N
Are launch sites in proximity to coastline? Y
Do launch sites keep certain distance away from cities? Y